# Assignment 3acCognitive Modelling

Dit is onderdeel van de derde opdracht voor Cognitive Modelling, assignment 3 bestaat in totaal uit 3 onderdelen:

* Assignment 3a (43 punten)
* Assignment 3b (21 punten)
* __Assignment 3c (23 punten)__

__Let op__: Opdracht 3b en 3c werken alleen in Google Colab!

Geef antwoorden in blokken met code of met tekst.Gebruik voor antwoorden met tekst de ">" voor blockquotes en geef bij elke vraag ook __kort uitleg__ als hier om wordt gevraagd. __Let op__: soms staan er meerdere vragen bij een onderdeel, lees de tekst dus nauwkeurig. 

Sla het uiteindelijke notebook op met jullie studentnummers en achternamen in de filenaam: `studentnummer_achternaam_opdrachtnummer`.

<br>

---

### Voorbereiding stap 1. De Colab environment

Koppel je notebook aan een GPU: ga naar **Edit > Notebook settings** of **Runtime > Change runtime type** en selecteer GPU as Hardware accelerator. 

![screenshot](https://drive.google.com/uc?id=1QsJXUPPL0eEoWHI7a-A1_F42djdgCcBL)

and daarna:

![screenshot](https://drive.google.com/uc?id=1ODvXSxF7OBy9blmZfC1Yk870uw80c6YW)

Nu moet je jouw Google Drive koppelen aan deze omgeving zodat je daarvan kan lezen en naar kan schrijven. Het zal misschien nodig zijn om een key in te voeren. Klik daarvoor op de link die verschijnt. Die brengt je naar een google pagina, en copy paste dan je key hier in het notebook. 


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/'Files for Colab'

Check even of alle files er zijn die je nodig hebt:

In [ ]:
ls

<br>

---

### Voorbereiding stap 2. Installeer Vizdoom ☕

Nu moeten we Vizdoom en een paar dependencies installeren op de Colab. Deze twee stappen kunnen enkele minuten duren (tijd voor koffie). Let op dat als je de colab instance een tijdje niet gebruikt, je deze stap opnieuw moet doen! Net als de key for google drive geven.

In [ ]:
## to get vizdoom to work 
## https://stackoverflow.com/questions/50667565/how-to-install-vizdoom-using-google-colab 
## takes a few minutes.... 
%%bash
# Install deps from https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux
apt-get update

apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip

# Boost libraries
apt-get install libboost-all-dev
# Lua binding dependencies
apt-get install liblua5.1-dev

En dan nu Vizdoom (ongeveer 5 minuten)

In [ ]:
# this takes a few minutes more.. 
!pip install vizdoom

<br>

---

### Voorbereiding stap 3. Setting up the Game environment

In [ ]:
# TF 1 code, just got deprecated! We go to TF1 for this exercise
%tensorflow_version 1.x
import tensorflow as tf        # Deep Learning library
print(tf.__version__)

import numpy as np              # Handle matrices
from vizdoom import *           # Doom Environment
import random                   # Handling random number generation
import time                     # Handling time calculation
from skimage import transform   # Help us to preprocess the frames
from collections import deque   # Ordered collection with ends
import matplotlib.pyplot as plt # Display graphs

import warnings                 # This ignore all the warning messages that are normally printed during the training
warnings.filterwarnings('ignore') 

In [ ]:
from IPython.display import Image
Image('Images/doom.png')

<br>

---

## De regels van het spel 🎮
Nu we alle libraries hebben, kunnen we de regels van het spel bepalen. Doom wordt bepaald door:
  - Een  `configuration file` "basic.cfg" daar in vinden alle opties (frame size, mogelijke handelingen)
  - Een `scenario file`: "basic.wad" deze genereerd het correcte scenario (ruimte, monsters etc, komt bij vizdoom) .

Open beide files met een text editor en bestudeer ze kort, kijk of je de belangrijkste elementen kan ontcijferen (en later mogelijk veranderen). Het spel ziet er als volgt uit:

__De omgeving:__
- Een monster staat op een willekeurige plek aan de overkant van het veld 
- De speler heeft drie mogelijke acties: naar links, naar rechts en schieten 
- 1 hit is genoeg om het monster uit te schakelen
- Een episode is over als het monster is geraakt of door een timeout (300).

__De rewards:__
- -5 punten voor het schieten van kogel  
- -1 punt voor elke tijdstap (energy consumption)
- +100 punten voor het uitschakelen van het monster uitschakelen

In [ ]:
# Hier definieren we de omgeving

def create_environment():
    game = DoomGame()
    
    # Load the correct configuration
    game.load_config("basic.cfg")
    
    # Load the correct scenario (in our case basic scenario)
    game.set_doom_scenario_path("basic.wad")
    
    ## we cant watch the game perform on collab
    game.set_window_visible(False)
    game.init()
    
    # Here our possible actions
    left = [1, 0, 0]
    right = [0, 1, 0]
    shoot = [0, 0, 1]
    possible_actions = [left, right, shoot]
    
    return game, possible_actions

In [ ]:
game, possible_actions = create_environment()

<br>

---

### Preprocessing ⚙️

Preprocessing is een belangrijke stap, omdat we de complexiteit van onze data willen verminderen om zo de rekentijd die nodig is voor training te verminderen. In termen van leren kan je deze stap zien als een leraar die de leerling alleen vertelt in welke richting te kijken. Minimale supervision, maar dit helpt de leerling wel door de aandacht te vestigen in de juiste richting.

__Preprocessing stappen:__
- Grijstinten maken van alle frames (omdat kleur geen belangrijke informatie toevoegt). Dit wordt al gedaan door het configuratiebestand (!).
- Het scherm bijsnijden (in ons geval verwijderen we het dak omdat het geen informatie bevat)
- Het normaliseren van de pixelwaarden 
- Het verkleinen van het preprocessed frame.

Dit resulteert in een 84x84 grayscale frames.

In [ ]:
"""
    preprocess_frame:
    Take a frame, resize it from
         _________________
        |                 |        __________  
        |                 |       |          |
        |                 |       |          |
        |                 |       |          |
        |_________________|  to   |__________|
        
    Normalize it,     
    return preprocessed_frame
    
    """
def preprocess_frame(frame):
    # Greyscale frame already done in our vizdoom config
    # x = np.mean(frame,-1)
    
    # Crop the screen (remove the roof because it contains no information)
    cropped_frame = frame[30:-10,30:-30]
    
    # Normalize Pixel Values
    normalized_frame = cropped_frame/255.0
    
    # Resize
    preprocessed_frame = transform.resize(normalized_frame, [84,84])
    
    return preprocessed_frame

Image('Images/preproc.png')

<br>

---

### Stacked Frames 🕖🕗🕘🕙

Als input states gaan we dus nu geen single frames gebruiken, maar een stapeltje (stacked) frames. Het stapelen van frames is erg belangrijk omdat het een gevoel van beweging/tijd meegeeft aan het neurale netwerk. Eerst preprocessen we het frame, vervolgens voegen we het frame toe aan een *deque* dat automatisch het oudste frame verwijdert, dan bouwen we de state (S) van deze frames.

Dit is hoe de functie werkt:
- Voor de start state voeren we 4 keer hetzelfde begin frame in
- Bij elke tijdsstap voegen we het nieuwe frame toe aan de deque, en wordt het oudste frame weggehaald. Deze nieuwe stack is dan de nieuwe state
- Enzovoort
- Als een episode voorbij is maken we een nieuwe stapel met 4 nieuwe frames (omdat we ons in een nieuwe episode bevinden).

Hier is nog een meer in depth artikel over stacked frames:  <a href="https://danieltakeshi.github.io/2016/11/25/frame-skipping-and-preprocessing-for-deep-q-networks-on-atari-2600-games/">  Frame skipping and preprocessing</a>.

In [ ]:
stack_size = 4 # We stack 4 frames

# Initialize deque with zero-images one array for each image
stacked_frames  =  deque([np.zeros((84,84), dtype=np.int) for i in range(stack_size)], maxlen=4) 

def stack_frames(stacked_frames, state, is_new_episode):
    # Preprocess frame
    frame = preprocess_frame(state)
    
    if is_new_episode:
        # Clear our stacked_frames
        stacked_frames = deque([np.zeros((84,84), dtype=np.int) for i in range(stack_size)], maxlen=4)
        
        # Because we're in a new episode, copy the same frame 4x
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        
        # Stack the frames
        stacked_state = np.stack(stacked_frames, axis=2)
        
    else:
        # Append frame to deque, automatically removes the oldest frame
        stacked_frames.append(frame)

        # Build the stacked state (first dimension specifies different frames)
        stacked_state = np.stack(stacked_frames, axis=2) 
    
    return stacked_state, stacked_frames

<br>

---

### Initialiseren van hyperparameters 🎛
In dit deel stellen we de verschillende hyperparameters in. Eerst definieren we de hyperparameters van het neurale netwerk:

In [ ]:
### MODEL HYPERPARAMETERS
state_size = [84,84,4]     
action_size = game.get_available_buttons_size()    # 3 possible actions: left, right, shoot
learning_rate =  0.002      # Alpha (aka learning rate)

### ❓ __Q1 (2 punten)__
Waarom is de input voor het model, `State_size`,  een matrix van 84x84x4 ?

> *Antwoord*

---

## Deep Q-learning Neural Network model bouwen 🧠

Dit is de structuur van het Deep Q-learning model (zie ook de afbeelding hieronder):
- We nemen de stacked frames as input 
- Deze input gaat door 3 convnets
- De data wordt platgeslagen (flattened), en gaat dan door twee fully connected layers. 

In [ ]:
Image('Images/model.png')

Bestudeer de code onder vraag 3 bij `class DQNetwork`, kijk in hoeverre je de structuur zoals hier boven beschreven terug kan lezen. Het is nu niet belangrijk dat alle details helemaal begrijpt, maar wel de grote lijnen. 

<br>

### ❓__Q2. Network Questions (4 punten)__

1. Hoeveel nodes heeft de laatse laag, en wat representeert deze laag?
2. Welke activatie functie gebruikt het netwerk?
3. En welke loss functie wordt er geminimalizeerd?

> *Antwoord*

<br>

Als het goed is heb je bij Computer Vision alles over Convnets geleerd en ook bij andere vakken over ANNs. Maar om toch even kort terug naar de theorie. Waarom gebruiken we hier niet een simpel one-layer network, of waarom geen Q-tables zoals we hiervoor hebben gedaan? De reden is de grootte van de state space.


### ❓__Q3. (2 punten)__ 
Hoe groot is de state space (na pre-processing) en hoeveel state, action pairs zijn er dus? 

> *Antwoord*

<br>

Precies! Dat is al heel erg groot en dit spel is nog steeds erg beperkt. Stel je voor dat je nog door de ruimte kan bewegen etc. We zagen al dat het grotere Frozen Lake van 8x8 lastig was voor Q-tables. Dit is niet te doen. Convnets kunnen de agent de state space laten doorgronden en laten generalizeren.

Een convnet kan gebruik maken van spatiele relaties: denk bijvoorbeeld aan dat als het monster twee stappen links van jou staat, je altijd LEFT, LEFT, SHOOT zou moeten doen, ongeacht of het monster helemaal links staat of in het midden. Als je deze relaties kan gebruiken om te leren is dat dus heel handig. Een Q-table aanpak zou voor state monster_links en state monster_midden geheel opnieuw moeten leren wat de beste actie is. 

Meer lezen over convolutional nets: 
https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53

In [ ]:
class DQNetwork:
    def __init__(self, state_size, action_size, learning_rate, name='DQNetwork'):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        
        with tf.variable_scope(name):
            # We create the placeholders
            # *state_size means that we take each elements of state_size in tuple hence is like if we wrote
            # [None, 84, 84, 4]
            self.inputs_ = tf.placeholder(tf.float32, [None, *state_size], name="inputs")
            self.actions_ = tf.placeholder(tf.float32, [None, 3], name="actions_")
            
            # Remember that target_Q is the R(s,a) + y*max Qhat(s', a')
            self.target_Q = tf.placeholder(tf.float32, [None], name="target")
            
            """
            First convnet:       CNN
            BatchNormalization:  ELU
            """
            # Input is 84x84x4
            self.conv1 = tf.layers.conv2d(inputs = self.inputs_,
                                         filters = 32,
                                         kernel_size = [8,8],
                                         strides = [4,4],
                                         padding = "VALID",
                                          kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                         name = "conv1")
            
            self.conv1_batchnorm = tf.layers.batch_normalization(self.conv1,
                                                   training = True,
                                                   epsilon = 1e-5,
                                                     name = 'batch_norm1')
            
            self.conv1_out = tf.nn.elu(self.conv1_batchnorm, name="conv1_out")  # --> [20, 20, 32]
            
            """
            Second convnet:      CNN
            BatchNormalization:  ELU
            """
            self.conv2 = tf.layers.conv2d(inputs = self.conv1_out,
                                 filters = 64,
                                 kernel_size = [4,4],
                                 strides = [2,2],
                                 padding = "VALID",
                                kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                 name = "conv2")
        
            self.conv2_batchnorm = tf.layers.batch_normalization(self.conv2,
                                                   training = True,
                                                   epsilon = 1e-5,
                                                     name = 'batch_norm2')

            self.conv2_out = tf.nn.elu(self.conv2_batchnorm, name="conv2_out") # --> [9, 9, 64]
            
            """
            Third convnet:       CNN
            BatchNormalization:  ELU
            """
            self.conv3 = tf.layers.conv2d(inputs = self.conv2_out,
                                 filters = 128,
                                 kernel_size = [4,4],
                                 strides = [2,2],
                                 padding = "VALID",
                                kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                 name = "conv3")
        
            self.conv3_batchnorm = tf.layers.batch_normalization(self.conv3,
                                                   training = True,
                                                   epsilon = 1e-5,
                                                     name = 'batch_norm3')

            self.conv3_out = tf.nn.elu(self.conv3_batchnorm, name="conv3_out") # --> [3, 3, 128]
            self.flatten = tf.layers.flatten(self.conv3_out) # --> [1152]
            
            self.fc = tf.layers.dense(inputs = self.flatten,
                                      units = 512,
                                      activation = tf.nn.elu, 
                                      kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                      name="fc1")
            
            self.output = tf.layers.dense(inputs = self.fc,
                                          kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                          units = 3,
                                          activation=None)

            # Q is our predicted Q value.
            self.Q = tf.reduce_sum(tf.multiply(self.output, self.actions_), axis=1)
            
            # The loss is the difference between our predicted Q_values and the Q_target
            # Sum(Qtarget - Q)^2
            self.loss = tf.reduce_mean(tf.square(self.target_Q - self.Q))
            
            self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)

In [ ]:
# Reset the graph
tf.reset_default_graph()

# Instantiate the DQNetwork
DQNetwork = DQNetwork(state_size, action_size, learning_rate)
# ignore warnings about TF 2.0

<br>

---

### De hyperparameters voor de training 🎛
In dit deel voegen we de trainingshyperparameters toe:

In [ ]:
### TRAINING HYPERPARAMETERS
total_episodes = 501          # Total episodes for training
max_steps = 100               # Max possible steps in an episode (let op max is 300 in game)
batch_size = 64               # size of batch used for replay

# Exploration parameters for epsilon greedy strategy
explore_start = 1.0           # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001           # exponential decay rate for exploration prob

# Q learning hyperparameters
gamma = 0.95                  # Discounting rate (zie lectures Temporal Difference learning)

### MEMORY HYPERPARAMETERS
pretrain_length = batch_size  # Number of experiences stored in the Memory when initialized for the first time
memory_size = 1000000         # Number of experiences the Memory can keep

### MODIFY THIS TO FALSE IF YOU JUST WANT TO SEE THE TRAINED AGENT
training = True

## TURN THIS TO TRUE IF YOU WANT TO RENDER THE ENVIRONMENT
episode_render = False

<br>

---

### Experience Replay 🔁
Nu we ons neurale netwerk hebben gemaakt, gaan we de Experience Replay-methode implementeren. Eerst maken we een deque Memory-object. Een deque (dubbele wachtrij) is een gegevenstype dat het oudste element verwijdert telkens wanneer er een nieuw element wordt toegevoegd.

In [ ]:
class Memory():
    def __init__(self, max_size):
        self.buffer = deque(maxlen = max_size)
    
    def add(self, experience):
        self.buffer.append(experience)
    
    def sample(self, batch_size):
        buffer_size = len(self.buffer)
        index = np.random.choice(np.arange(buffer_size),
                                size = batch_size,
                                replace = False)
        
        return [self.buffer[i] for i in index]

Hier zullen we het probleem aanpakken dat de agents beginnen met een volledig leeg geheugen: we vullen ons geheugen vooraf in door willekeurige acties te ondernemen en de ervaring op te slaan (*state, action, reward, new_state*).

In [ ]:
memory = Memory(max_size = memory_size)    # Instantiate memory
game.new_episode()                         # Render the environment

for i in range(pretrain_length):
    
    # If it's the first step
    if i == 0:
        state = game.get_state().screen_buffer                             # First we need a state
        state, stacked_frames = stack_frames(stacked_frames, state, True)  # Stack the frame
    
    action = random.choice(possible_actions)  # Random action
    reward = game.make_action(action)         # Get the rewards
    done = game.is_episode_finished()         # Check if the episode is finished
    
    # If we're dead
    if done:
        next_state = np.zeros(state.shape)                                 # We finished the episode
        memory.add((state, action, reward, next_state, done))              # Add experience to memory
        game.new_episode()                                                 # Start a new episode        
        state = game.get_state().screen_buffer                             # First we need a state
        state, stacked_frames = stack_frames(stacked_frames, state, True)  # Stack the frame
        
    else: 
        next_state = game.get_state().screen_buffer                                  # Get the next state
        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False) # Stack the frame
        memory.add((state, action, reward, next_state, done))                        # Add experience to memory
        state = next_state                                                           # Our state is now the next_state

<br>

---

### Tensorboard 📊

We gebruiken deze functie om de data van de training weg te schrijven. Voor meer info zie deze <a href="https://www.youtube.com/embed/eBbEDRsCmv4">tutorial</a>

In [ ]:
writer = tf.summary.FileWriter("/tensorboard/dqn/1")    # Setup TensorBoard Writer
tf.summary.scalar("Loss", DQNetwork.loss)               # Losses
write_op = tf.summary.merge_all()

Hieronder definiëren we de keuzefunctie van de agent. We implementeren een  klassieke e-greedy keuzeregel met exponentiële afname in exploratie. Een nieuwigheid is dat er een ondergrens is voor de parameter **explore_stop**.

In [ ]:
"""
This function will do the part
With ϵ select a random action atat, otherwise select at=argmaxaQ(st,a)
"""
def predict_action(explore_start, explore_stop, decay_rate, decay_step, state, actions):
    ## EPSILON GREEDY STRATEGY
    # Choose action a from state s using epsilon greedy.
    exp_exp_tradeoff = np.random.rand()                 # First we randomize a number

    # Here we'll use an improved version of our epsilon greedy strategy used in Q-learning notebook
    explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)
    
    if (explore_probability > exp_exp_tradeoff):
        action = random.choice(possible_actions)        # Explore
        
    else:
        # Get action from Q-network (exploitation), estimate the Qs values state
        Qs = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: state.reshape((1, *state.shape))})
        
        # Take the biggest Q value (= the best action)
        choice = np.argmax(Qs)
        action = possible_actions[int(choice)]
                
    return action, explore_probability

<br>

---

## Training 🏃‍♂️

Ons algoritme:
<br>
* Initialiseer de weigths van het netwerk
* Initialiseer de game omgeving
* Initialiseer de decay rate voor e-greedy
<br>
* **For** episode to max_episode **do** 
    * Make new episode
    * Set step to 0 
    * Observe the first state $s_0$ 
    <br>
    * **While** step < max_steps **do**:
        * Increase decay_rate (decreas decay)
        * With $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s_t,a)$
        * Execute action $a_t$ and observe reward $r_{t+1}$ and new state $s_{t+1}$
        * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
        * Sample random mini-batch from $D$: $<s, a, r, s'>$
        * Set $\hat{Q} = r$ if the episode ends at $+1$, otherwise set $\hat{Q} = r + \gamma \max_{a'}{Q(s', a')}$
        * Make a gradient descent step with loss $(\hat{Q} - Q(s, a))^2$

Je ziet dat dit eigenlijk precies hetzelfde is als we eerder in het FrozenLake Q-net hadden geimplementeerd, alleen zitten er nu dus wel wat lagen tussen.

<br>

### ❓ __Q4.a (7 punten)__

Je gaat hieronder de agent trainen in 500 episodes met de huidige set hyperparameters, dit kan wel ongeveer 10 minuten duren, dus tijd om even pauze te nemen en de benen te strekken. Bekijk en rapporteer na afloop het gedrag van de agent tijdens het leerproces. Dit kan op twee manieren:

1. Aan de hand van de score die de agent behaalt tijdens het leren;
2. Aan de hand van hoe de agent het spel speelt (visueel).

__1. Score__
  
  * Tijdens het trainen schrijft het script de output weg in **test.txt** die kan worden gebruikt voor visualisatie van de leercurve(s). Deze wordt dus in je Google Drive geschreven. Geef deze file een andere naam die specifiek is voor setttings van het netwerk dat je traint, zo kan je later modelen vergelijken.
  * Maak twee plots op basis van deze data: de leercurve met de score van de agent tijdens het leren, de kans op een exploratie. Zet in de grafieken de gemiddelde punten en explore probabilities voor bins van 10 episodes om de leercurven te illustreren en rapporteer ook het totaal aantal punten. Beschrijf wat je ziet.

__2. Visualisatie__ 

* De code hieronder slaat het model om de 50 episodes `model_XX.ckpt.` op in de directory `models`. Dit is de staat van het model na XX episodes trainen. De filenames specificeren dus weer het model. 

  Met de code onder vraag 5 kan je de boven genoemde outputfiles gebruiken om naar gedrag van de agent te kijken, gegeven de weights in het netwerk na een een XX aantal leer episodes. Deze code laat slechts één game zien (één episode), dus doe dit misschien een aantal keer omdat het anders net zo kan uitvallen dat de game begint dat de agent recht voor de alien staat en meteen neerschiet. Dat is goed maar vertelt weinig over het gedrag. 

  Naast deze ene game, slaat deze code ook de sequentie van states als lijst van screenshots op in `trajectory`. Deze `trajectory` kan je in een animatie te maken door ze achter elkaar te plakken. Je kan deze  bekijken, en ook opslaan als mp4.
  
* Beschrijf het verschil in gedrag in drie verschillende fasen (episode 50, 150 en 500). Wat voor soort fouten zie je? Sla de vier .mp4 bestanden van deze drie fases op en __lever de bestanden in__.

_Let op: voor sommigen werkt de functie_ `save_path = saver.save(sess, "./models/model_{}.ckpt".format(episode)` _niet altijd goed. Het is niet duidelijk waarom, maar dan kan je dit deel van de opdracht helaas niet helemaal doen, laat dit weten aan je TA._

In [ ]:
f= open("test.txt","a+") # <- verander deze filename!

In [ ]:
# Saver will help us to save our model
saver = tf.train.Saver()

if training == True:
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())     # Initialize the variables 
        decay_step = 0                                  # Initialize the decay rate (that will use to reduce epsilon)
        game.init()                                     # Init the game

        for episode in range(total_episodes):
            step = 0                                    # Set step to 0
            episode_rewards = []                        # Initialize the rewards of the episode
            game.new_episode()                          # Make a new episode and observe the first state
            state = game.get_state().screen_buffer
            
            # Remember that stack frame function also call our preprocess function.
            state, stacked_frames = stack_frames(stacked_frames, state, True)

            while step < max_steps:
                step += 1
                decay_step +=1 # Increase decay_step
                
                # Predict the action to take and take it
                action, explore_probability = predict_action(explore_start, explore_stop, decay_rate, decay_step, state, possible_actions)

                reward = game.make_action(action)        # Do the action
                done = game.is_episode_finished()        # Look if the episode is finished
                episode_rewards.append(reward)           # Add the reward to total reward
                total_reward = np.sum(episode_rewards)   # Get the total reward of the episode
                
                # If the game is finished
                if done:
                    next_state = np.zeros((84,84), dtype=np.int) # the episode ends so no next state
                    next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

                    step = max_steps  # Set step = max_steps to end the episode                    
                    memory.add((state, action, reward, next_state, done))

                else:
                    next_state = game.get_state().screen_buffer             # Get the next state
                    next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
                    memory.add((state, action, reward, next_state, done))   # Add experience to memory
                    state = next_state                                      # st+1 is now our current state

                ### LEARNING PART            
                batch = memory.sample(batch_size)   # Obtain random mini-batch from memory
                states_mb = np.array([each[0] for each in batch], ndmin=3)
                actions_mb = np.array([each[1] for each in batch])
                rewards_mb = np.array([each[2] for each in batch]) 
                next_states_mb = np.array([each[3] for each in batch], ndmin=3)
                dones_mb = np.array([each[4] for each in batch])

                target_Qs_batch = []

                 # Get Q values for next_state 
                Qs_next_state = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: next_states_mb})
                
                # Set Q_target = r if the episode ends at s+1, otherwise set Q_target = r + gamma*maxQ(s', a')
                for i in range(0, len(batch)):
                    terminal = dones_mb[i]

                    # If we are in a terminal state, only equals reward
                    if terminal:
                        target_Qs_batch.append(rewards_mb[i])
                        
                    else:
                        target = rewards_mb[i] + gamma * np.max(Qs_next_state[i])
                        target_Qs_batch.append(target)
                        
                targets_mb = np.array([each for each in target_Qs_batch])

                loss, _ = sess.run([DQNetwork.loss, DQNetwork.optimizer],
                                    feed_dict={DQNetwork.inputs_: states_mb,
                                               DQNetwork.target_Q: targets_mb,
                                               DQNetwork.actions_: actions_mb})

                # Write TF Summaries
                summary = sess.run(write_op, feed_dict={DQNetwork.inputs_: states_mb,
                                                        DQNetwork.target_Q: targets_mb,
                                                        DQNetwork.actions_: actions_mb})
                writer.add_summary(summary, episode)
                writer.flush()
            
            print('Episode: {}'.format(episode),
                              'Total reward: {}'.format(total_reward),
                              'Training loss: {:.4f}'.format(loss),
                              'Explore P: {:.4f}'.format(explore_probability))
            
            f.write('Episode: {}'.format(episode) + "\t" +
                              'Total reward: {}'.format(total_reward) + "\t"+
                              'Explore P: {:.4f}'.format(explore_probability) + "\t"+
                               'Training loss: {:.4f}'.format(loss) + "\n")
                              
            # Save model every 50 episodes
            if episode % 50 == 0:
                save_path = saver.save(sess, "./models/model_{}.ckpt".format(episode))
                print("Model Saved")

In [ ]:
f.close()

In [ ]:
# TO DO plot

> *Antwoord*

<br>

---

### ❓Q4.b (6 punten)
Probeer nu de prestaties van het model te verbeteren en voer bovenstaande stappen nog eens uit. Dit kan je ondertussen op heel veel manieren doen, je kan bijvoorbeeld veranderingen maken aan de hyperparameters, exploratie regel, replay etc. 

* Leg uit waarom je verwachtte dat jouw aanpassing het beter zou maken
* Laat zien of het werkte op basis van traingsgrafiek (stap 1 hierboven)
* Kijk goed naar het gedrag na 500 episodes leren of je daar iets van kan afleiden (stap 2 hierboven). 

Het aantal trainings episodes is technische gezien een hyperparameter, maar die bedoelen we dus niet :)

**Let op:** 
1. Elke keer voordat je een nieuwe training uitvoert met nieuwe hyperparameters moet je de game opnieuw initialseren: <br>
``
game, possible_actions = create_environment()
`` <br>

2. Je moet ook de weights van het netwerk resetten (model resetten door opnieuw initialisatiestappen uit te voeren), begin bij <br> ``tf.reset_default_graph()
DQNetwork = DQNetwork(state_size, action_size, learning_rate)`` (net boven het kopje *'De hyperparameters voor de training'*).<br>
3. Reset ook het geheugen (bij het kopje *'Experience replay'*)

Dan is het geheugen van de CoLab instantie (RAM) ook weer leeg en het netwerk klaar om te gaan leren. Ook nu gaat het om je idee, niet meteen of het netwerk ook beter leert, maar dat is natuurlijk wel een mooie uitkomst!

In [ ]:
# TO DO

> *Antwoord*

<br>

---

### ❓__Q5 (2 Punten)__
Bestudeer onderstaande code, hoe verschilt de decision policy van de gedrags simulatie met die van de training agent?

> *Antwoord*

In [ ]:
## Dit codeblock en het volgende heb je ook nodig om het tweede deel van Q4.a te beantwoorden
trajectory = []

with tf.Session() as sess:
    
    game, possible_actions = create_environment()
    totalScore = 0
    
    # Load the model
    saver.restore(sess, "./models/model_350.ckpt") #<- change this to check behavior after different learning episodes
    game.init()
    for i in range(1):
        
        done = False
        game.new_episode()
        
        state = game.get_state().screen_buffer
        state, stacked_frames = stack_frames(stacked_frames, state, True)
            
        while not game.is_episode_finished():
            # Get Q values for each action
            Qs = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: state.reshape((1, *state.shape))})
            
            # Select action
            choice = np.argmax(Qs)
            action = possible_actions[int(choice)]
            
            game.make_action(action)
            done = game.is_episode_finished()
            score = game.get_total_reward()
            
            if done:
                break  
                
            else:
                #print("step")
                next_state = game.get_state().screen_buffer
                trajectory.append(next_state)
                next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
                state = next_state
                
        score = game.get_total_reward()
        print("Score: ", score)
    game.close()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc

rc('animation', html='html5')
%matplotlib inline

fig, ax = plt.subplots()

def animate(i):
    ax.imshow(trajectory[i])
    return ax

gif = animation.FuncAnimation(fig, animate, frames=len(trajectory))
gif
gif.save('DOOM_350.mp4', fps=5, dpi=200) # comment out gif.save and rerun block to see amin in line, or just check your mp4. 

<br>

---

### Smart Exploration & Beyond..
Je hebt hier gezien dat we weer e-greedy gebruikte om met de explore exploit trade-off om te gaan. Maar we hadden eerder ook al wel gezien dat dit niet altijd de slimste of beste manier van exploratie is. We hebben UCB besproken die niet bezochte states een exploratie bonus geeft, en die bonus neemt af naarmate er meer over een staat geleerd is. Zo kom je niet meer terug bij "slechte" states.

Nu willen we niet alle mogelijk states per se bezoeken, van sommige kunnen we misschien al direct voorspellen dat deze niets gaan opleveren. In deze context is Curiosity Based Learning dus heel interessant. Kijk nogmaals naar [Curiosity driven learning](https://towardsdatascience.com/curiosity-driven-learning-made-easy-part-i-d3e5a2263359) voor een goede uitleg. 

Het gaat nu te ver om hier dieper op in te gaan maar ik hoop dat door de basis te snappen van DQNs en dat dit je nu al een beter idee hebt hoe deze algoritmes werken. Probeer ook zeker zelf verschillende games van Vizdoom uit, of andere games zoals Space invaders. Er zijn eindeloos veel tutorials te vinden op het internet.